In [ ]:
import gc
import math
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

import scipy.stats as st
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth, DBSCAN, OPTICS

import infoStructure as ins
import helpers as hp
import clustering as cl
import display as dp
import importlib #importlib.reload(foo)

> Point to the directories (output for the processed mat files and where the mat files reside

In [ ]:
INPUTDir = '' # the directory where the all the patients are (RS1000, RS10001...)
OUTPUTDir = '' # the directory where to save all the created files
# also, the variables to get from .mat files used all along the study
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue', 'darkorange', 'gold', 
                              'tomato', 'crimson', 'darkslategray', 'springgreen', 'chocolate'])
titles = ['vectorRRKLD', 'vectorFAmpKLD', 'vectorUFAmpKLD', 'vectorCorrKLD'] 
feat_dict = None
df_ALL = None

> Grab all the patients directory that you need, and the specific features from the 34

<span style="color:red">Just needed to extract information. If needed, remove the tripple quotes</span>
```python
""" commented code """
```

> /!\ ATENTION: in the "addAllPatientsInfoV4" script, the most suitable variable for scalable data is to_hdf = True, because it won't saturate RAM, even though, it's bigger than feather in ROM and takes more time on loading. 

> Because of this, the recommended way to do this is to extract [100] patients (as quantities) and since they are taken randomly from the source, this would help, afterwards around 75% from this info should be extracted to do the further calculations

In [ ]:
importlib.reload(ins)
# files to save (of 5, and 10 and 50 ... patients inside)
quantities = [5, 20, 50, 100] #[5, 10, 50, 100, 200] # max num of patients = 524
## randomly
ins.addAllPatientsInfoV4(INPUTDir, titles, quantities, OUTPUTDir, to_hdf=True)

## in order, starting from N # of directory, qty = number of directories
#qty = 10
#start_from = 0
#feat_dict = ins.addAllPatientsInfoV3(INPUTDir, titles, qty, start_from)

** Grab the information previously treated and saved as a feather file **

In [ ]:
name_of_file = '100_f32.h5' #'550_32.feather'

# if the information has been grabed from previous cell (from patients input files)
#df_ALL = hp.convertDictInDF(feat_dict)
df_ALL = hp.readFileToPandas(OUTPUTDir + name_of_file)

In [ ]:
df_ALL.info()

In [ ]:
df_ALL.head()

** According to the quantity of the memory available, grab randomly the rows for the study of the data **
> 0.22 (22%) for a 16Go RAM memory avaliable computer, for one model

> 0.16 for a 16Go RAM memory available computer, for running several models (like GMM but several times to get the best BIC score)

In [ ]:
df_DIV, indexes = hp.getRandomRows(df_ALL, 0.5)
df_DIV, df_info = hp.cleanDF(df_DIV, ['paths', 'voie_num']) # divide DF between pure info and data

### Preprocessing

In [ ]:
df_DIV = cl.cleanData(df_DIV, 'mean') # impute non available data in the columns using a strategy (mean, median, most_frequent)

> Dependign if PCA wants to be applied, run one or the other, in this step, removing outliers and normalization takes place

> Besides, in order to do a hard removal for the difficult "vectorUFAmpKLD" feature removal, choose v3=True for the runOutNormV2, else, if just wanted it to be handled with the meanshift appraoch, use v3=False

In [ ]:
df_nout, Xnorm, std, indexes2 = cl.runOutNormV2(df_DIV, indexes, threshold=20, threshold_hard=0.01, v3=True)
## for PCA run this one
#df_nout, Xnorm, Xpca, dfPca, titPca, pca, std, indexes2 = cl.runOutNormPCAV2(df_DIV, indexes, threshold=20, threshold_hard=0.01, cols_hard=[0,2])

> declare which matrix of data should be used for the models according to the previous choice

In [ ]:
X = Xnorm # Xnorm or Xpca

> print data as obtained from preprocessing, 
 ```python
if Xpca, the "titPca" variable should be passed,
else titles of columns should be passed (['vectorRRKLD', 'vectorFAmpKLD', 'vectorUFAmpKLD', 'vectorCorrKLD'])
```

In [ ]:
#dp.printPCAScatter(df_nout, titles)
del df_nout; gc.collect()
# del dfPca; gc.collect()

### Model

```python 
if used v3=False for the preprorcessing step (runOutNormV2 function), components should be around 8 and 10
else components are around 6 and 8
```

In [ ]:
gmm = GaussianMixture(n_components=7, covariance_type='full', random_state=0).fit(X)
#gmm10 = GaussianMixture(n_components=10, covariance_type='full').fit(X)

> ** TO PRINT**

```python
# for 2d prinring of all clusters together (6 images), use: 
dp.printPCAGMM(gmm_to_print, X, titles, color_iter)
# for 2d printing of one cluster, use next: 
# ( gmm, gmm.predict(X), cluster #, column_1_to_display, column_2_to_display, color_of_cluster, titles )
fig, ax = plt.subplots(1, 1)
dp.plotOneGMMCluster(ax, X, predicted, 0, 0, 1, 'turquoise', titles)
# for 3d printing of onw cluster, use next: 
# ( 3dfig, gmm, gmm.predict(X), cluster #, column_1_to_display, column_2_to_display, column_3_to_display, color_of_cluster, titles)
ax = plt.axes(projection='3d') # just once!!
dp.plotOneGMMCluster3D(ax, X, predicted, 2, 0, 1, 2, 'turquoise', titles)
# to print all 6 possible combinations 
dp.plotAll2DGMMs(X, predicted, 1, 'turquoise', titles)
```

> <span style="color:red">put one plot by cell, if not, dynamic display wont appear for each plot</span>

In [ ]:
gmm_to_print = gmm
predicted = gmm_to_print.predict(X)
print(hp.getRepresentativeness(gmm_to_print, X, predicted))

**<span style="color:red">Extract the desired data from a cluster, complemented with its information</span>**

In [ ]:
quantity_of_rows = 5
cluster = 0
desired_data = hp.getFromClusterInfo(X, predicted, quantity_of_rows, indexes2, cluster, df_info, titles)
desired_data

** For each cluster, and for different features, the next cell chould be modified taking into accunt the variables mentioned above ** 

*Since just one cluster colored by 2 cells considering 4 dimensions, 2 plots must be donne to show each cluster according the 4 dimensions in a 3d plot*

In [ ]:
%matplotlib inline 
dp.plotAll2DGMMs(X, predicted, 0, 'red', titles)

In [ ]:
%matplotlib inline 
dp.plotAll2DGMMs(X, predicted, 1, 'red', titles)

----------------------------------------------------

### 3D MANIPULATION

In [ ]:
# to show a dinamic view "%matplotlib notebook" , if not desired, use "%matplotlib inline"
%matplotlib inline 
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 0, 0, 1, 2, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 0, 2, 3, 1, 'red', titles)
plt.show(); #plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 1, 2, 0, 3, 'red', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 2, 1, 2, 3, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 2, 2, 1, 3, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
# to show a dinamic view "%matplotlib notebook" , if not desired, use "%matplotlib inline"
%matplotlib inline 
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 3, 0, 2, 3, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 4, 0, 1, 2, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 5, 0, 2, 3, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 6, 0, 2, 3, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

In [ ]:
ax = plt.axes(projection='3d')
dp.plotOneGMMCluster3D(ax, X, predicted, 1, 0, 1, 2, 'turquoise', titles)
plt.show(); plt.clf(); plt.close(); gc.collect()

### Understand de BIC (Bayesian Information Criterion) Optional
> If runned, take into account the percentage of data from the total, for a 16Go computer it will only support 15% og the 120 million rows with 4 features in float32

In [ ]:
"""
n_components_range = range(5, 12)
best_gmm, bic, cv_types = cl.getBestGMMUsingBIC(X, n_components_range, ['full'], 0.1)
bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue', 'darkorange', 'gold', 'tomato', 'crimson', 'darkslategray', 'springgreen', 'chocolate'])
dp.plotBICScores(bic, cv_types, color_iter, n_components_range)
"""